In [1]:
from functions.load_data import load_CIFAR_data, load_Google_Digit_Data, load_MNIST_data, shuffle_data
from functions.attacks import attackTestSet, attackTestSetBatch
from functions.models import simple_relu_model, simple_tropical_model, TropicalResNet50, ReLUResNet50
from custom_layers.initializers import BimodalBinaryInitializer, BimodalNormalInitializer, Triangular
from tensorflow.keras import losses, initializers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from custom_layers.tropical_layers import TropEmbedMaxMin, TropConv2D
from custom_layers.initializers import BimodalNormalInitializer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Dropout, Reshape, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time

In [2]:
from custom_layers.tropical_layers import TropEmbedMaxMin, TropConv2D
from custom_layers.initializers import BimodalNormalInitializer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Flatten, Conv2D, Dropout, Reshape, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time

In [3]:
# -- load SVNH data -- 
x_train, x_test, y_train, y_test = load_Google_Digit_Data(desired_classes=[i for i in range(10)])
x_train = x_train.reshape(-1,32,32,3)
x_test = x_test.reshape(-1,32,32,3)

In [3]:
# -- load MNIST data -- 
x_train, x_test, y_train, y_test = load_MNIST_data(desired_classes=[i for i in range(10)])
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [14]:
def conv_trop_then_relu_3layer_model(x_train, 
                       y_train, 
                       num_epochs = 10,
                       batch_size = 64, 
                       verbose = 1, 
                       initializer_trop = BimodalNormalInitializer(stddev=1,high=5.5, low=-4.5),
                       initializer_relu = initializers.RandomNormal(mean=0.5, stddev=1., seed=0),
                       final_layer_activation = 'softmax', 
                       training_loss = 'categorical_crossentropy',
                       lam = 0):
    start_time = time.time() 
    #in_channels = x_train.shape[-1]
    model = Sequential([Flatten(),
                        TropEmbedMaxMin(1024, initializer_w = initializer_trop, lam=lam),
                        Reshape((1024, 1)),
                        Conv1D(32, 3, activation='relu'),
                        MaxPooling1D(2),                            
                        Conv1D(32, 3, activation='relu'),
                        MaxPooling1D(2),
                        Conv1D(32, 3, activation='relu'),
                        Flatten(),
                        Dense(64, activation='relu', kernel_initializer = initializer_relu),
                        Dense(10, activation=final_layer_activation, kernel_initializer = initializer_relu)])
    model.compile(optimizer='adam', loss=training_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=num_epochs,batch_size=batch_size, verbose=verbose)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tropical model built. Elapsed time: {elapsed_time:.2f} seconds | {elapsed_time/60:.2f} minutes.")
    
    #model.build(input_shape=(32, 28, 28, 1))
    #model.summary()
    return model

conv_trop_then_relu_3layer_model(x_train=x_train, y_train=y_train, lam = 0.01)

Epoch 1/10
 31/938 [..............................] - ETA: 3:16 - loss: 6363.5205 - accuracy: 0.0953

KeyboardInterrupt: 

In [ ]:
def conv_trop_then_relu_3layer_model(x_train, 
                       y_train, 
                       num_epochs = 10,
                       batch_size = 64, 
                       verbose = 1, 
                       initializer_trop = BimodalNormalInitializer(stddev=1,high=5.5, low=-4.5),
                       initializer_relu = initializers.RandomNormal(mean=0.5, stddev=1., seed=0),
                       final_layer_activation = 'softmax', 
                       training_loss = 'categorical_crossentropy',
                       lam = 0):
    start_time = time.time() 
    #in_channels = x_train.shape[-1]
    model = Sequential([Flatten(),
                        TropEmbedMaxMin(1024, initializer_w = initializer_trop, lam=lam),
                        Reshape((32, 32, 1)),
                        Conv2D(32, (3,3), activation='relu'),
                        MaxPooling2D((2,2)),                            
                        Conv2D(32, (3,3), activation='relu'),
                        MaxPooling1D((2,2)),
                        Conv2D(32,(3,3), activation='relu'),
                        Flatten(),
                        Dense(64, activation='relu', kernel_initializer = initializer_relu),
                        Dense(10, activation=final_layer_activation, kernel_initializer = initializer_relu)])
    model.compile(optimizer='adam', loss=training_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=num_epochs,batch_size=batch_size, verbose=verbose)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tropical model built. Elapsed time: {elapsed_time:.2f} seconds | {elapsed_time/60:.2f} minutes.")
    #model.build(input_shape=(32, 28, 28, 1))
    #model.summary()
    return model

conv_trop_then_relu_3layer_model(x_train=x_train, y_train=y_train, lam = 0.01)

In [5]:
def conv_trop_then_relu_3layer_model(x_train, 
                       y_train, 
                       num_epochs = 10,
                       batch_size = 64, 
                       verbose = 1, 
                       initializer_trop = initializers.random_normal,
                       initializer_relu = initializers.RandomNormal(mean=0.5, stddev=1., seed=0),
                       final_layer_activation = 'softmax', 
                       training_loss = 'categorical_crossentropy',
                       lam = 0.01):
    start_time = time.time() 
    #in_channels = x_train.shape[-1]
    model = Sequential([TropConv2D(32, window_size = [1, 1, 1, 1], initializer_w = initializer_trop, lam=lam),
                        MaxPooling2D((2,2)),
                        Conv2D(32, (3,3), activation='relu'),
                        MaxPooling2D((2,2)),
                        Conv2D(32, (3,3), activation='relu'),
                        Flatten(),
                        Dense(64, activation='relu'),
                        Dense(10, activation=final_layer_activation, kernel_initializer = initializer_relu)])
    model.compile(optimizer='adam', loss=training_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=num_epochs,batch_size=batch_size, verbose=verbose)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tropical model built. Elapsed time: {elapsed_time:.2f} seconds | {elapsed_time/60:.2f} minutes.")
    #model.build(input_shape=(32, 28, 28, 1))
    #model.summary()
    return model

trop_model = conv_trop_then_relu_3layer_model(x_train=x_train, y_train=y_train)

Epoch 1/10
1145/1145 [==============================] - 114s 97ms/step - loss: 1.9064 - accuracy: 0.3359
Epoch 2/10
 472/1145 [===========>..................] - ETA: 1:25 - loss: 1.5919 - accuracy: 0.4622

KeyboardInterrupt: 

In [21]:
def conv_trop_then_relu_3layer_model(x_train, 
                       y_train, 
                       num_epochs = 10,
                       batch_size = 64, 
                       verbose = 1, 
                       initializer_trop = initializers.random_normal,
                       initializer_relu = initializers.RandomNormal(mean=0.5, stddev=1., seed=0),
                       final_layer_activation = 'softmax', 
                       training_loss = 'categorical_crossentropy',
                       lam = 0.01):
    start_time = time.time() 
    #in_channels = x_train.shape[-1]
    model = Sequential([TropConv2D(1000, initializer_w = initializer_trop, lam=lam),
                        Flatten(),
                        TropEmbedMaxMin(10, initializer_w = initializer_trop, lam=lam),
                        Dense(10, activation=final_layer_activation, kernel_initializer = initializer_relu)])
    model.compile(optimizer='adam', loss=training_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=num_epochs,batch_size=batch_size, verbose=verbose)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tropical model built. Elapsed time: {elapsed_time:.2f} seconds | {elapsed_time/60:.2f} minutes.")
    #model.build(input_shape=(32, 28, 28, 1))
    #model.summary()
    return model

troppy_model = conv_trop_then_relu_3layer_model(x_train=x_train, y_train=y_train, lam = 0.01)

Epoch 1/10
938/938 [==============================] - 292s 306ms/step - loss: 9.5132 - accuracy: 0.1868
Epoch 2/10
938/938 [==============================] - 428s 456ms/step - loss: 4.8339 - accuracy: 0.3557
Epoch 3/10
938/938 [==============================] - 356s 380ms/step - loss: 4.6360 - accuracy: 0.4139
Epoch 4/10
408/938 [============>.................] - ETA: 2:41 - loss: 4.5347 - accuracy: 0.4418

KeyboardInterrupt: 

In [6]:
def conv_relu_then_trop_3layer_model(x_train, 
                       y_train, 
                       num_epochs = 10,
                       batch_size = 64, 
                       verbose = 1,
                       initializer_relu = initializers.RandomNormal(mean=0.5, stddev=1., seed=0),
                       final_layer_activation = 'softmax', 
                       training_loss = 'categorical_crossentropy',
                       initializer_w = BimodalNormalInitializer(stddev=1,high=5.5, low=-4.5),
                       lam=0.0):
    start_time = time.time() 
    input_shape = x_train[0].shape
    model = Sequential([TropConv2D(filters = 32),
                        MaxPooling2D((2, 2)),                            
                        Conv2D(32, (3, 3), activation='relu'),
                        MaxPooling2D((2, 2)),
                        Conv2D(32, (3, 3), activation='relu'),
                        Flatten(),
                        #TropEmbedMaxMin(64, initializer_w = initializer_w, lam=lam),
                        Dense(10, activation=final_layer_activation, kernel_initializer = initializer_relu)])
    model.compile(optimizer='adam', loss=training_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=num_epochs,batch_size=batch_size, verbose=verbose)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tropical model built. Elapsed time: {elapsed_time:.2f} seconds | {elapsed_time/60:.2f} minutes.")
    #model.build(input_shape=(32, 28, 28, 1))
    #model.summary()
    return model

troppy_model = conv_relu_then_trop_3layer_model(x_train=x_train, y_train=y_train, lam = 0.01)

Epoch 1/10
938/938 [==============================] - 100s 106ms/step - loss: 0.5720 - accuracy: 0.8334
Epoch 2/10
938/938 [==============================] - 97s 104ms/step - loss: 0.2306 - accuracy: 0.9344
Epoch 3/10
938/938 [==============================] - 95s 101ms/step - loss: 0.1728 - accuracy: 0.9524
Epoch 4/10
938/938 [==============================] - 97s 103ms/step - loss: 0.1502 - accuracy: 0.9603
Epoch 5/10
938/938 [==============================] - 96s 103ms/step - loss: 0.1366 - accuracy: 0.9645
Epoch 6/10
938/938 [==============================] - 96s 103ms/step - loss: 0.1278 - accuracy: 0.9668
Epoch 7/10
938/938 [==============================] - 95s 101ms/step - loss: 0.1157 - accuracy: 0.9699
Epoch 8/10
938/938 [==============================] - 98s 105ms/step - loss: 0.1077 - accuracy: 0.9737
Epoch 9/10
938/938 [==============================] - 100s 106ms/step - loss: 0.1046 - accuracy: 0.9730
Epoch 10/10
938/938 [==============================] - 105s 112ms/step 

In [9]:
def conv_relu_then_trop_3layer_model(x_train, 
                       y_train, 
                       num_epochs = 10,
                       batch_size = 64, 
                       verbose = 1,
                       initializer_relu = initializers.RandomNormal(mean=0.5, stddev=1., seed=0),
                       final_layer_activation = 'softmax', 
                       training_loss = 'categorical_crossentropy',
                       initializer_w = BimodalNormalInitializer(stddev=1,high=5.5, low=-4.5),
                       lam=0.0):
    start_time = time.time() 
    input_shape = x_train[0].shape
    model = Sequential([TropConv2D(filters = 32),
                        MaxPooling2D((2, 2)),                            
                        Conv2D(32, (3, 3), activation='relu'),
                        #MaxPooling2D((2, 2)),
                        #Conv2D(32, (3, 3), activation='relu'),
                        Flatten(),
                        
                        Dense(10, activation=final_layer_activation, kernel_initializer = initializer_relu)])
    model.compile(optimizer='adam', loss=training_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=num_epochs,batch_size=batch_size, verbose=verbose)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tropical model built. Elapsed time: {elapsed_time:.2f} seconds | {elapsed_time/60:.2f} minutes.")
    #model.build(input_shape=(32, 28, 28, 1))
    #model.summary()
    return model

troppy_model = conv_relu_then_trop_3layer_model(x_train=x_train, y_train=y_train, lam = 0.01)

Epoch 1/10
938/938 [==============================] - 66s 67ms/step - loss: 1.7219 - accuracy: 0.5028
Epoch 2/10
938/938 [==============================] - 61s 65ms/step - loss: 0.5071 - accuracy: 0.8455
Epoch 3/10
938/938 [==============================] - 72s 77ms/step - loss: 0.3193 - accuracy: 0.9072
Epoch 4/10
938/938 [==============================] - 133s 141ms/step - loss: 0.2481 - accuracy: 0.9313
Epoch 5/10
938/938 [==============================] - 78s 83ms/step - loss: 0.2103 - accuracy: 0.9429
Epoch 6/10
938/938 [==============================] - 62s 66ms/step - loss: 0.2122 - accuracy: 0.9416
Epoch 7/10
938/938 [==============================] - 60s 64ms/step - loss: 0.2020 - accuracy: 0.9431
Epoch 8/10
938/938 [==============================] - 60s 64ms/step - loss: 0.1806 - accuracy: 0.9509
Epoch 9/10
938/938 [==============================] - 62s 66ms/step - loss: 0.1709 - accuracy: 0.9536
Epoch 10/10
938/938 [==============================] - 60s 64ms/step - loss: 0.1

In [10]:
# -- TROPICAL: attack model -- 
trop_x_test_att = attackTestSetBatch(troppy_model, x_test, y_test, epsilon=0.1, loss_object=losses.CategoricalCrossentropy(), modelName='Tropical')

# -- TROPICAL: evaluate -- 
# - before attack -
pre_trop_preds = np.argmax(troppy_model.predict(x_test), axis=1)
y_tester = np.argmax(y_test, axis=1)
pre_correct_index = np.equal(pre_trop_preds, y_tester)
pre_correct_accuracy = np.average(pre_correct_index)
troppy_model.evaluate(x_test, y_test)

# - after attack - 
post_trop_preds = np.argmax(troppy_model.predict(trop_x_test_att), axis=1)
post_correct_index = np.equal(post_trop_preds, y_tester)
post_correct_accuracy = np.average(post_correct_index)
troppy_model.evaluate(trop_x_test_att, y_test)

# - number maintaining prediction - 
maintained_correct_pred = np.average(np.equal(post_trop_preds[pre_correct_index], y_tester[pre_correct_index]))
print(pre_correct_accuracy, post_correct_accuracy, maintained_correct_pred)

Tropical model 0% attacked.
Tropical model 40% attacked.
Tropical model 80% attacked.
Tropical attack complete. Elapsed time: 154.07 seconds /// 2.57 minutes.
313/313 [==============================] - 3s 11ms/step - loss: 1.2608 - accuracy: 0.5974
0.9595 0.5974 0.6224075039082856


In [24]:
def conv_relu_3layer_model(x_train, 
                       y_train, 
                       num_epochs = 10,
                       batch_size = 64, 
                       verbose = 1,
                       initializer_relu = initializers.random_normal,
                       final_layer_activation = 'softmax', 
                       training_loss = 'categorical_crossentropy',
                       initializer_w = BimodalNormalInitializer(stddev=1,high=5.5, low=-4.5),
                       lam=0.0):
    start_time = time.time() 
    input_shape = x_train[0].shape
    model = Sequential([Conv2D(32, (1, 1), activation='relu'),
                        MaxPooling2D((2, 2)),                            
                        Conv2D(64, (3, 3), activation='relu'),
                        MaxPooling2D((2, 2)),                            
                        Conv2D(64, (3, 3), activation='relu'),
                        Flatten(),
                        Dense(64, activation='relu', kernel_initializer = initializer_relu),
                        Dense(10, activation=final_layer_activation, kernel_initializer = initializer_relu)])
    model.compile(optimizer='adam', loss=training_loss, metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=num_epochs,batch_size=batch_size, verbose=verbose)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Tropical model built. Elapsed time: {elapsed_time:.2f} seconds | {elapsed_time/60:.2f} minutes.")
    #model.build(input_shape=(32, 28, 28, 1))
    #model.summary()
    return model

relu_model = conv_relu_3layer_model(x_train=x_train, y_train=y_train, lam = 0.01)

Epoch 1/10
1145/1145 [==============================] - 119s 103ms/step - loss: 1.0009 - accuracy: 0.6780
Epoch 2/10
1145/1145 [==============================] - 132s 116ms/step - loss: 0.5018 - accuracy: 0.8523
Epoch 3/10
1145/1145 [==============================] - 134s 117ms/step - loss: 0.4266 - accuracy: 0.8730
Epoch 4/10
1145/1145 [==============================] - 137s 120ms/step - loss: 0.3785 - accuracy: 0.8878
Epoch 5/10
1145/1145 [==============================] - 138s 120ms/step - loss: 0.3486 - accuracy: 0.8978
Epoch 6/10
1145/1145 [==============================] - 144s 125ms/step - loss: 0.3244 - accuracy: 0.9029
Epoch 7/10
1145/1145 [==============================] - 137s 119ms/step - loss: 0.3000 - accuracy: 0.9109
Epoch 8/10
1145/1145 [==============================] - 128s 112ms/step - loss: 0.2833 - accuracy: 0.9154
Epoch 9/10
1145/1145 [==============================] - 134s 117ms/step - loss: 0.2673 - accuracy: 0.9202
Epoch 10/10
1145/1145 [=======================

In [25]:
# -- RELU: fit model --
#relu_model = conv_relu_3layer_model(x_train=x_train, y_train=y_train)

# -- RELU: attack model -- 
relu_x_test_att = attackTestSetBatch(relu_model, x_test, y_test,  epsilon=0.1, loss_object=losses.CategoricalCrossentropy(), num_steps = 20, modelName='ReLU')

# -- RELU: evaluate --
# - before attack -
pre_relu_preds = np.argmax(relu_model.predict(x_test), axis=1)
y_tester = np.argmax(y_test, axis=1)
pre_correct_index = np.equal(pre_relu_preds, y_tester)
pre_correct_accuracy = np.average(pre_correct_index)
relu_model.evaluate(x_test, y_test)

# - after attack - 
post_relu_preds = np.argmax(relu_model.predict(relu_x_test_att), axis=1)
post_correct_index = np.equal(post_relu_preds, y_tester)
post_correct_accuracy = np.average(post_correct_index)
relu_model.evaluate(relu_x_test_att, y_test)

# - number maintaining prediction - 
maintained_correct_pred = np.average(np.equal(post_relu_preds[pre_correct_index], y_tester[pre_correct_index]))
print(pre_correct_accuracy, post_correct_accuracy, maintained_correct_pred)

ReLU model 0% attacked.
ReLU attack complete. Elapsed time: 994.96 seconds /// 16.58 minutes.
814/814 [==============================] - 9s 11ms/step - loss: 21.6798 - accuracy: 0.0254
0.8943223724646588 0.02539182544560541 0.026588204974013145


In [21]:

# Create a ResNet-50 model
model = ResNet50(input_shape=(32, 32, 3), num_classes=10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#model.summary()

# Define batch size and number of epochs
batch_size = 64
epochs = 10

# Train the model
model.fit(x_train, np.argmax(y_train, axis=1), batch_size=batch_size, epochs=epochs)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, np.argmax(y_test, axis=1))
print(f"Test accuracy: {test_acc}")


Epoch 1/10
   5/1145 [..............................] - ETA: 21:49 - loss: 4.8526 - accuracy: 0.1312

KeyboardInterrupt: 

In [19]:
np.argmax(y_train, axis=1)

array([9, 7, 0, ..., 9, 4, 7], dtype=int64)

Total params: 20,637,416
Trainable params: 20,623,208
Non-trainable params: 14,208